In [3]:
import torch 
import torch.nn.functional as F
import matplotlib as plt
%matplotlib inline

# read all the words 
words = words = open('/Users/m/dev/makemore/names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [2]:
len(words)

32033

In [4]:
# build the vocabulary of chars and the mappings to/from ints
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
vocab_size = len(stoi)

In [5]:
# build the dataset
block_size = 3 # context length: how many characters do we take to predict the next one?

def build_dataset(words):  
  X, Y = [], []
  for w in words:

    #print(w)
    context = [0] * block_size
    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      #print(''.join(itos[i] for i in context), '--->', itos[ix])
      context = context[1:] + [ix] # crop and append

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape, Y.shape)
  return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = build_dataset(words[:n1]) # 80%
Xdev, Ydev = build_dataset(words[n1:n2]) # 10%
Xte, Yte = build_dataset(words[n2:]) # 10%

torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


In [14]:
C = torch.randn((27,2)) #initial random init
C[5]

tensor([ 1.0089, -0.4192])

In [60]:
emb = C[X]
emb.shape

torch.Size([228146, 3, 2])

In [61]:
W1 = torch.randn((6,100))
b1 = torch.randn(100)

In [ ]:
h = torch.tanh(emb.view(32,6) @ W1 + b1)
h.shape

In [26]:
W2 = torch.randn((100, 27))
b2 = torch.randn(27)

In [27]:
logits = h @ W2 + b2
logits.shape

torch.Size([32, 27])

In [30]:
counts = logits.exp()
counts.shape

torch.Size([32, 27])

In [32]:
prob = counts / counts.sum(1, keepdim=True)
prob.shape

torch.Size([32, 27])

In [37]:
loss = -prob[torch.arange(32), Y].log().mean()
loss

tensor(20.0160)

In [ ]:
### getting it all tidied up for trainig and inference ### 

In [84]:
Xtr.shape, Ytr.shape

(torch.Size([182580, 3]), torch.Size([182580]))

In [6]:
n_embd = 10 # the dimensionality of the char embedding vectors
n_hidden = 200 # the number of neurons in the hidden MLP laer

g = torch.Generator().manual_seed(2147483647) # for reproducibility
C = torch.randn((vocab_size, n_embd), generator=g)
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) * 0.02
# b1 = torch.randn(n_hidden, generator=g) * 0.01
W2 = torch.randn((n_hidden, vocab_size), generator=g) * 0.01 # prevent high init values
b2 = torch.randn(vocab_size, generator=g) * 0 # prevent high init values

# BatchNorm parameters
bngain = torch.ones((1, n_hidden))
bnbias = torch.zeros((1, n_hidden))
bnmean_running = torch.zeros((1, n_hidden))
bnstd_running = torch.ones((1, n_hidden))

parameters = [C, W1, W2, b2, bngain, bnbias]
print(sum(p.nelement() for p in parameters)) # number of parameters in total
for p in parameters:
  p.requires_grad = True

12097


In [7]:
# training
# same optimization as last time
max_steps = 200000
batch_size = 32
lossi = []

for i in range(max_steps):
  
  # minibatch construct
  ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
  Xb, Yb = Xtr[ix], Ytr[ix] # batch X,Y
  
  # forward pass
  emb = C[Xb] # embed the characters into vectors
  embcat = emb.view(emb.shape[0], -1) # concatenate the vectors
  # Linear layer
  hpreact = embcat @ W1 #+ b1 # hidden layer pre-activation
  # BatchNorm layer
  # -------------------------------------------------------------
  bnmeani = hpreact.mean(0, keepdim=True)
  bnstdi = hpreact.std(0, keepdim=True)
  hpreact = bngain * (hpreact - bnmeani) / bnstdi + bnbias
  with torch.no_grad():
    bnmean_running = 0.999 * bnmean_running + 0.001 * bnmeani
    bnstd_running = 0.999 * bnstd_running + 0.001 * bnstdi
  # -------------------------------------------------------------
  # Non-linearity
  h = torch.tanh(hpreact) # hidden layer
  logits = h @ W2 + b2 # output layer
  loss = F.cross_entropy(logits, Yb) # loss function
  
  # backward pass
  for p in parameters:
    p.grad = None
  loss.backward()
  
  # update
  lr = 0.1 if i < 100000 else 0.01 # step learning rate decay
  for p in parameters:
    p.data += -lr * p.grad

  # track stats
  if i % 10000 == 0: # print every once in a while
    print(f'{i:7d}/{max_steps:7d}: {loss.item():.4f}')
  lossi.append(loss.log10().item())

      0/ 200000: 3.3147
  10000/ 200000: 2.1550
  20000/ 200000: 2.3059
  30000/ 200000: 2.4165
  40000/ 200000: 1.9505
  50000/ 200000: 2.2925
  60000/ 200000: 2.3911
  70000/ 200000: 2.0873
  80000/ 200000: 2.3733
  90000/ 200000: 2.1597
 100000/ 200000: 1.9075
 110000/ 200000: 2.2214
 120000/ 200000: 1.8906
 130000/ 200000: 2.4627
 140000/ 200000: 2.3773
 150000/ 200000: 2.2281
 160000/ 200000: 1.8634
 170000/ 200000: 1.8009
 180000/ 200000: 2.0639
 190000/ 200000: 1.9266


In [10]:
# evalutaion 
@torch.no_grad() # do not track gradients
def split_loss(split):
    x,y = {
        'train': (Xtr, Ytr),
        'val': (Xdev, Ydev),
        'test': (Xte, Yte),
    }[split]

    emb = C[x] # # (N, block_size, n_embd)
    emb_cat = emb.view(emb.shape[0], -1) # concat into (N, block_size * n_embd)
    
    h_pre_act = emb_cat @ W1 + b1 # hidden layer pre-activation
    # batch norm
    h_pre_act = bngain * (h_pre_act - bnmean_running) / bnstd_running + bnbias

    h = torch.tanh(h_pre_act) #(N, n_hidden)
    logits = h @ W2 + b2 # #(N, vocab_size)
    loss = F.cross_entropy(logits, y)
    print(split, loss.item())

split_loss('train')
split_loss('val')

train 2.0584306716918945
val 2.1071255207061768


In [160]:
# sample from the model
g = torch.Generator().manual_seed(2147483647 + 10)

for _ in range(20):
    
    out = []
    context = [0] * block_size # initialize with all ...
    while True:
      # forward pass
      emb = C[torch.tensor([context])] # (1,block_size,d)
      h = torch.tanh(emb.view(1, -1) @ W1 + b1)
      logits = h @ W2 + b2
      probs = F.softmax(logits, dim=1)
      # sample from distribution
      ix = torch.multinomial(probs, num_samples=1, generator=g).item()
      context = context[1:] + [ix]
      out.append(ix)
      # break if we come across '.' token
      if ix == 0:
        break
    
    print(''.join(itos[i] for i in out))

chapahzaquriqlifrimrishthtylellassiesmahnbefledsahcidgquifnellaishckaiirdesleggydh.
mussiddessanishlileasviviquinceligijarixlifradelissangelizahirtetthleyshershuashbudrissyannaqqorizsophilynnalairosbek.
khadlivosgodssidorahmrynegastleythranskaelishanslyviadalif.
qwelleghvikhridgprissslahkkelistabbrighdre.
khleskariffatdseveghn.
qscatorriah.
junyannakingsa.
liliszes.
sukellistimurgscingstzy.
balyselichasdanyssahkynleghanvicadmbrykuniahlireqmisabrishakyrtz.
imensrighgndfontightdng.
gabblumichvikharahppimshakwebduvivannagapfmichawdhelistaj.
juviedrickherslyusspenza.
savanjumwin.
ikvannakhazklimhkyaz.
lylishmaybil.
jevtalessiverezleycajwne.
pebyagazliszandrtynahisuschimamdyssadershewdy.
lizzensuisherssamakyan.
avryshidanaiseassideb.
